In [ ]:
import re
import os

In [432]:
# global variables
eng_bigrams = None
fr_bigrams = None
ital_bigrams = None

eng_unigrams = None
fr_unigrams = None
ital_unigrams = None

In [433]:
# Start here 

# open all the input training files
# if you wish to change the path of the files to open, change them on the following three lines
input_eng = open("/LangId.train.English")
input_fr = open("/LangId.train.French")
input_ital = open("/LangId.train.Italian")

# create english, french, and italian letter bigram counts

eng_bigrams = file_to_bigrams(input_eng)
fr_bigrams = file_to_bigrams(input_fr)
ital_bigrams = file_to_bigrams(input_ital)

# create english, french, and italian letter unigram counts
eng_unigrams = create_unigrams(input_eng)
fr_unigrams = create_unigrams(input_fr)
ital_unigrams = create_unigrams(input_ital)


# open input testing file
# if you wish to change your input test or output solution file names, do so on the following lines.
input_test = open("/LangId.test")
# open output testing file (write to this)
output_test = open("/wordLangId.out", "w+")

line_count = 1

# iterate through input file and write the most likely language for each line in output file
for line in input_test.readlines():
    # choose_language returns a string: "English", "French", or "Italian"
    lang = choose_language(line)
    # write it to output file along with line number
    l = str(line_count) + lang + "\n"
    output_test.write(l)
    line_count = line_count + 1

input_eng.close()
input_fr.close()
input_ital.close()
input_test.close()
output_test.close()

In [434]:
def create_unigrams(infile):
    # so it can be read again
    infile.seek(0)
    lines = []
    for line in infile.readlines():
            # strip all punctionation from each lowercased word
        no_punc = re.sub("[@#$%&.,:;!]+", '', line.lower())
        lines.append(no_punc)

    unigrams = {}
    for line in lines:
        words = line.split()
        for i in range(0, len(words) - 1):
            ug = (words[i])
            if ug not in unigrams.keys():
                unigrams[ug] = 1  
            else:
                unigrams[ug] = unigrams[ug] + 1
        
    return unigrams 

In [435]:
# calls create bigrams
# use this function when creating the bigram models from the training files 
def file_to_bigrams(infile):
    lines = []
    for line in infile.readlines():
            # strip all punctionation from each lowercased word
        no_punc = re.sub("[@#$%&.,:;!]+", '', line.lower())
        lines.append(no_punc)

    bigrams = {}
    for line in lines:
        words = line.split()
        for i in range(0, len(words) - 1):
            bg = ((words[i], words[i+1]))
            if bg not in bigrams.keys():
                bigrams[bg] = 1  
            else:
                bigrams[bg] = bigrams[bg] + 1
        
    return bigrams 


In [436]:
# trains the model for a language
# creates a dictionary of bigrams with keys = (bigram tuple), value = count
# i.e. one entry might be ('t', 'h'): 3
# takes just one line as input
def create_bigrams(line):
    
    # create bigram ditionary
    bigrams = {}
    words = line.split()
    for i in range(0, len(words) - 1):
        bg = ((words[i], words[i+1]))
        if bg not in bigrams.keys():
            bigrams[bg] = 1  
        else:
            bigrams[bg] = bigrams[bg] + 1
    return bigrams 


In [437]:
# calculate the probabilities of each language for the input line
# and returns a string form of the language that is most likely
def choose_language(line):

    # calculate probabilities of line being the given language
    eng_prob = calc_probability(line, eng_bigrams, eng_unigrams)
    fr_prob = calc_probability(line, fr_bigrams, fr_unigrams)
    ital_prob = calc_probability(line, ital_bigrams, ital_unigrams)

    # find highest likelihood language
    highest_prob = max(eng_prob, fr_prob, ital_prob)
    # return the language with the highest likelihood
    if highest_prob == eng_prob:
        return " English"
    
    if highest_prob == fr_prob:
        return " French"

    if highest_prob == ital_prob:
        return " Italian"


    # shouldn't be here....
    return ":("

In [ ]:
def calc_probability(line, bigrams, unigrams):

    line_bigrams = create_bigrams(line.lower())
    
    probabilities = {}

    for bigram in line_bigrams.keys():
        word1 = bigram[0]
        # calculating probabilities using no smoothing
        if bigram in bigrams and word1 in unigrams:
            probabilities[bigram] = bigrams[bigram] / unigrams[word1] 
        else:
            # out-of-vocabulary: 
            probabilities[bigram] = 0

    sum_prob = 0
    for k in probabilities.keys():
        sum_prob = sum_prob + probabilities[k]
    
    return sum_prob